<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark/challenges/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 1
##  Implement INGESTION process
- Set up path in the "lake"
  - !mkdir -p /content/lake/bronze

- Read data from API https://api.carrismetropolitana.pt/
  - Endpoints:
    - vehicles
    - lines
    - municipalities
  - Use StructFields to enforce schema

- Transformations
  - vehicles
    - create "date" extracted from "timestamp" column (format: hh24miss)

- Write data as PARQUET into the BRONZE layer (/content/lake/bronze)
  - Partition "vehicles" by "date" column
  - Paths:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# CHALLENGE 2
##  Implement CLEANSING process
- Set up path in the "lake"
  - !mkdir -p /content/lake/silver

- Read data from BRONZE layer as PARQUET:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities

- Transformations
  - vehicles
    - rename "lat" and "lon" to "latitude" and "longitude" respectively
    - remove possible duplicates
    - remove rows when the column CURRENT_STATUS is null
    - remove any corrupted record
  - lines
    - remove duplicates
    - remove rows when the column LONG_NAME is null
    - remove any corrupted record
  - municipalities
    - remove duplicates
    - remove rows when the columns NAME or DISTRICT_NAME are null
    - remove any corrupted record

- Write data as PARQUET into the SILVER layer (/content/lake/silver)
  - Partition "vehicles" by "date"(created in the ingestion)
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities

# CHALLENGE 3
##  Implement ENRICH process
- Set up path in the "lake"
  - !mkdir -p /content/lake/gold

- Read data from SILVER layer
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities
  - Use StructFields to enforce schema

- Enrichment
  - Enrich vehicles dataset with information from the line and municipalities
    - join vehicles with lines and municipalities
      - select all columns from vehicles + lines.long_name (name: line_name, format:string) + municipalities.name (name: municipality_name, format: array)
      - Note that "municipalities.name" is an array

- Write data as PARQUET into the GOLD layer (/content/lake/gold)
  - Dataset name: vehicles_enriched
  - Partition "vehicles_enriched" by "date" column
  - Paths:
    - vehicles - path: /content/lake/gold/vehicles_enriched
  - Make sure there is only 1 single parquet created
  - Use overwrite as write mode

# CHALLENGE 4
##  Analyze data

- Query table "vehicles_enriched" in gold layer
- Aggregate data by municipality_name (array)
- Calculate:
  - count of vehicles (id) that pass through that municipality
  - sum speed of vehicles

Questions:
  - What are the top 3 municipalities by vehicles routes?
  - What are the top 3 municipalities with higher vehicle speed on average?


Tips:
- explode array into rows -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html

# Setting up PySpark

In [ ]:
%pip install pyspark

In [ ]:
!mkdir -p /content/lake/bronze

In [1]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests

class ETLFlow:
    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract_from_file(self, format: str, path: str, **kwargs) -> DataFrame:
        df = self.spark.read.format(format).load(path)
        return df

    def extract_from_api(self, url: str, schema: StructType = None):
      response = requests.get(url)
      rdd = spark.sparkContext.parallelize(response.json())
      if schema:
        df = spark.read.schema(schema).json(rdd)
      else:
        df = spark.read.json(rdd)
      return df

    def load(self, df: DataFrame, format: str, path: str, **kwargs) -> None:
        df.write.mode("overwrite").format(format).save(path)

class ETLTask(ETLFlow):

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def ingestion_vehicles(self):
      vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                                  StructField('block_id', StringType(), True),
                                  StructField('current_status', StringType(), True),
                                  StructField('id', StringType(), True),
                                  StructField('lat', FloatType(), True),
                                  StructField('line_id', StringType(), True),
                                  StructField('lon', FloatType(), True),
                                  StructField('pattern_id', StringType(), True),
                                  StructField('route_id', StringType(), True),
                                  StructField('schedule_relationship', StringType(), True),
                                  StructField('shift_id', StringType(), True),
                                  StructField('speed', FloatType(), True),
                                  StructField('stop_id', StringType(), True),
                                  StructField('timestamp', TimestampType(), True),
                                  StructField('trip_id', StringType(), True)])
      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)
      #self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles")
      self.write_partitioned_parquet(df=df, path="/content/lake/bronze/vehicles", partition_col="timestamp")


    def ingestion_lines(self):
      lines_schema = StructType([StructField("id", StringType(), True),
                                 StructField("short_name", StringType(), True),
                                 StructField("long_name", StringType(), True),
                                 StructField("municipalities", ArrayType(StringType()), True),
                                 StructField("localities", ArrayType(StringType()), True),
                                 StructField("routes", ArrayType(StringType()), True),
                                 StructField("patterns", ArrayType(StringType()), True),
                                 StructField("facilities", ArrayType(StringType()), True)])
      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/lines", schema=lines_schema)
      self.load(df=df, format="parquet", path="/content/lake/bronze/lines")


    def ingestion_municipalities(self):
        municipalities_schema = StructType([StructField("district_id", StringType(), True),
                                            StructField("district_name", StringType(), True),
                                            StructField("id", StringType(), True),
                                            StructField("name", StringType(), True),
                                            StructField("prefix", StringType(), True),
                                            StructField("region_id", StringType(), True),
                                            StructField("region_name", StringType(), True)])
        df = self.extract_from_api(url="https://api.carrismetropolitana.pt/municipalities", schema=municipalities_schema)
        self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")

    def cleansing_vehicles(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")

      #df = df.withColumn("date",date_format('timestamp',"HHmmss"))
      df = df.withColumnRenamed("lat", "latitude")\
                  .withColumnRenamed("lon", "longitude")
      df = df.drop_duplicates()
      df = df.dropna(subset=['CURRENT_STATUS'])
      self.write_partitioned_parquet(df=df, path="/content/lake/silver/vehicles", partition_col="timestamp")

    def cleansing_lines(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/lines")

      df = df.drop_duplicates()
      df = df.dropna(subset=['LONG_NAME'])
      self.load(df=df, format="parquet", path="/content/lake/silver/lines")

    def cleansing_municipalities(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/municipalities")

      df = df.drop_duplicates()
      #remove rows when the columns NAME or DISTRICT_NAME are null
      df = df.dropna(subset=['NAME', 'DISTRICT_NAME'])
      self.load(df=df, format="parquet", path="/content/lake/silver/municipalities")

    def write_partitioned_parquet(self, df: DataFrame, path: str, partition_col: str):

      # Convert the 'timestamp' column to a timestamp type before applying date_format
      df = df.withColumn(partition_col, from_unixtime(col(partition_col).cast("long")))
      df = df.withColumn(partition_col, date_format(partition_col, "HHmmss"))

      ## date(from_unixtime(1732305594))
      #df = df.withColumn("date", expr("date(timestamp)"))
      #self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles", partition_column="date")

      self.spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
      (df
        .coalesce(1) #make sure that there is only one parquet created
        .write
        .mode("overwrite")
        .partitionBy(partition_col)
        .format("parquet")
        .save(path))

    def enrich(self):
      vehicles = self.extract_from_file(format="parquet", path="/content/lake/silver/vehicles")
      lines = self.extract_from_file(format="parquet", path="/content/lake/silver/lines")
      municipalities = self.extract_from_file(format="parquet", path="/content/lake/silver/municipalities")

      #como é que eu enforço o esquema

      vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                                  StructField('block_id', StringType(), True),
                                  StructField('current_status', StringType(), True),
                                  StructField('id', StringType(), True),
                                  StructField('latitude', FloatType(), True),
                                  StructField('line_id', StringType(), True),
                                  StructField('longitude', FloatType(), True),
                                  StructField('pattern_id', StringType(), True),
                                  StructField('route_id', StringType(), True),
                                  StructField('schedule_relationship', StringType(), True),
                                  StructField('shift_id', StringType(), True),
                                  StructField('speed', FloatType(), True),
                                  StructField('stop_id', StringType(), True),
                                  StructField('timestamp', TimestampType(), True),
                                  StructField('trip_id', StringType(), True)])
      lines_schema = StructType([StructField("id", StringType(), True),
                                 StructField("short_name", StringType(), True),
                                 StructField("long_name", StringType(), True),
                                 StructField("municipalities", ArrayType(StringType()), True),
                                 StructField("localities", ArrayType(StringType()), True),
                                 StructField("routes", ArrayType(StringType()), True),
                                 StructField("patterns", ArrayType(StringType()), True),
                                 StructField("facilities", ArrayType(StringType()), True)])
      municipalities_schema = StructType([StructField("district_id", StringType(), True),
                                            StructField("district_name", StringType(), True),
                                            StructField("id", StringType(), True),
                                            StructField("name", StringType(), True),
                                            StructField("prefix", StringType(), True),
                                            StructField("region_id", StringType(), True),
                                            StructField("region_name", StringType(), True)])

      #Transformations' logic 1
      vehicles_enriched = vehicles.join(lines, vehicles.line_id == lines.id, "inner").select(vehicles["*"],lines["long_name"])

      #Transformations' logic 2
      # Step 2: Join with mapping DataFrame to replace IDs with names
      mapping_df = municipalities["id", "name"]
      mapping_df = mapping_df.withColumnRenamed("id","id_municipalities")

      # Step 1: Explode the `municipalities` column
      df1_exploded = lines.withColumn("id_exploded", explode(col("municipalities")))
      joined_df = df1_exploded.join(mapping_df, df1_exploded.id_exploded == mapping_df.id_municipalities, "left").select("id", "name")
      joined_df = joined_df.drop("id_municipalities")
      result_df = joined_df.groupBy("id").agg(collect_list("name").alias("municipality_names_array"))
      final_df = lines.join(result_df, "id", "left")
      final_df = final_df["id","municipality_names_array"]
      final_df = final_df.withColumnRenamed("id", "final_df_id")
      vehicles_enriched = vehicles_enriched.join(final_df, vehicles_enriched.line_id == final_df.final_df_id, "left")

      df_exploded3 = vehicles_enriched.withColumn("municipality_names_array_exploded", explode(col("municipality_names_array")))

      self.write_partitioned_parquet(df=df_exploded3, path="/content/lake/gold/vehicles", partition_col="timestamp") #perceber se o dataset tem a coluna timestamp

In [2]:
if __name__ == '__main__':

    # init spark
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master('local').appName('ETL Program').getOrCreate()

    print("Starting ETL program")
    etl = ETLTask(spark)

    # run tasks
    print("Running Task - Ingestion Vehicles")
    etl.ingestion_vehicles()

    print("Running Task - Ingestion lines")
    etl.ingestion_lines()

    print("Running Task - Ingestion municipalities")
    etl.ingestion_municipalities()

    print("Running Task - Cleansing Vehicles")
    etl.cleansing_vehicles()

    print("Running Task - Cleansing Lines")
    etl.cleansing_lines()

    print("Running Task - Cleansing municipalities")
    etl.cleansing_municipalities()


    etl.enrich()

    print("ETL program completed")

Starting ETL program
Running Task - Ingestion Vehicles
Running Task - Ingestion lines
Running Task - Ingestion municipalities
Running Task - Cleansing Vehicles
Running Task - Cleansing Lines
Running Task - Cleansing municipalities
ETL program completed


In [ ]:
spark.read.parquet("/content/lake/bronze/vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|      0|20241126-64010328...| IN_TRANSIT_TO|44|12660|38.767437|   4720|-9.100793|  4720_0_1|  4720_0|            SCHEDULED|113360234560|      0.0| 160027|4720_0_1|2600|220...|   215224|
|     90|20241126-64010054...| IN_TRANSIT_TO|44|12526| 38.70977|   4504|-8.945842|  4504_0_1|  4504_0|            SCHEDULED|121890234560|10.555555| 100335|4504_0_1|2600|213...|   215224|
|     56|20241126-64010152...| IN_TRANSIT_TO|44|12673| 38.57032| 

In [ ]:
spark.read.parquet("/content/lake/silver/vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|     56|20241126-64010152...| IN_TRANSIT_TO|44|12673| 38.57032|   4710|-8.905296|  4710_0_2|  4710_0|            SCHEDULED|113310234560|      0.0| 130803|4710_0_2|2600|220...|   114704|
|    180|UNAVAILABLE_BLOCK_ID| IN_TRANSIT_TO| 41|1269| 38.72345|   1502| -9.23503|  1502_0_2|  1502_0|            SCHEDULED|        1119|10.833333| 120276|1502_0_2_2130_215...|   114704|
|    208|       ESC_DU_EU1048| IN_TRANSIT_TO| 43|2368|38.618267| 

In [ ]:
spark.read.parquet("/content/lake/bronze/lines").show()

+----+----------+--------------------+--------------+--------------------+--------------------+--------------------+----------+
|  id|short_name|           long_name|municipalities|          localities|              routes|            patterns|facilities|
+----+----------+--------------------+--------------+--------------------+--------------------+--------------------+----------+
|1001|      1001|Alfragide (Estr S...|        [1115]|[Alfragide, Amado...|            [1001_0]|[1001_0_1, 1001_0_2]|        []|
|1002|      1002|Reboleira (Estaçã...|        [1115]|[Reboleira, Amado...|            [1002_0]|          [1002_0_3]|        []|
|1003|      1003|Amadora (Estação ...|        [1115]|[Amadora, Amadora...|            [1003_0]|[1003_0_1, 1003_0_2]|        []|
|1004|      1004|Amadora (Estação ...|        [1115]|[Amadora, Moinhos...|            [1004_0]|          [1004_0_3]|        []|
|1005|      1005|Amadora (Estação ...|        [1115]|[Amadora, Casal d...|[1005_0, 1005_1, ...|[1005_0_1

In [ ]:
spark.read.parquet("/content/lake/bronze/municipalities").show()

+-----------+-------------+----+--------------------+------+---------+----------------+
|district_id|district_name|  id|                name|prefix|region_id|     region_name|
+-----------+-------------+----+--------------------+------+---------+----------------+
|         07|        Évora|0712|        Vendas Novas|    19|    PT187|Alentejo Central|
|         11|       Lisboa|1101|            Alenquer|    20|    PT16B|           Oeste|
|         11|       Lisboa|1102|   Arruda dos Vinhos|    20|    PT16B|           Oeste|
|         11|       Lisboa|1105|             Cascais|    05|    PT170|             AML|
|         11|       Lisboa|1106|              Lisboa|    06|    PT170|             AML|
|         11|       Lisboa|1107|              Loures|    07|    PT170|             AML|
|         11|       Lisboa|1109|               Mafra|    08|    PT170|             AML|
|         11|       Lisboa|1110|              Oeiras|    12|    PT170|             AML|
|         11|       Lisboa|1111|

In [ ]:
!ls /content/lake/bronze/vehicles

spark.read.format("parquet").load("/content/lake/bronze/vehicles").count()

'timestamp=203553'  'timestamp=203617'	'timestamp=203634'  'timestamp=203651'	'timestamp=203708'
'timestamp=203554'  'timestamp=203618'	'timestamp=203635'  'timestamp=203652'	'timestamp=203709'
'timestamp=203555'  'timestamp=203619'	'timestamp=203636'  'timestamp=203653'	'timestamp=203710'
'timestamp=203600'  'timestamp=203620'	'timestamp=203637'  'timestamp=203654'	'timestamp=203711'
'timestamp=203601'  'timestamp=203621'	'timestamp=203638'  'timestamp=203655'	'timestamp=203712'
'timestamp=203603'  'timestamp=203622'	'timestamp=203639'  'timestamp=203656'	'timestamp=203713'
'timestamp=203604'  'timestamp=203623'	'timestamp=203640'  'timestamp=203657'	'timestamp=203714'
'timestamp=203605'  'timestamp=203624'	'timestamp=203641'  'timestamp=203658'	'timestamp=203715'
'timestamp=203607'  'timestamp=203625'	'timestamp=203642'  'timestamp=203659'	'timestamp=203716'
'timestamp=203609'  'timestamp=203626'	'timestamp=203643'  'timestamp=203700'	'timestamp=203717'
'timestamp=203610'  'timestamp

612

In [ ]:
!ls /content/lake/bronze/vehicles/timestamp=203650

part-00000-5a8099e4-69be-4027-aadd-4d15cbce08f4.c000.snappy.parquet


In [ ]:
!mkdir -p /content/lake/silver

In [ ]:
vehicles = spark.read.parquet("/content/lake/silver/vehicles")
lines = spark.read.parquet("/content/lake/silver/lines")
municipalities = spark.read.parquet("/content/lake/silver/municipalities")

In [ ]:
vehicles.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+
|     56|20241126-64010152...| IN_TRANSIT_TO|44|12673| 38.57032|   4710|-8.905296|  4710_0_2|  4710_0|            SCHEDULED|113310234560|      0.0| 130803|4710_0_2|2600|220...|   114704|
|    180|UNAVAILABLE_BLOCK_ID| IN_TRANSIT_TO| 41|1269| 38.72345|   1502| -9.23503|  1502_0_2|  1502_0|            SCHEDULED|        1119|10.833333| 120276|1502_0_2_2130_215...|   114704|
|    208|       ESC_DU_EU1048| IN_TRANSIT_TO| 43|2368|38.618267| 

In [ ]:
lines.show()

+----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  id|short_name|           long_name|      municipalities|          localities|              routes|            patterns|facilities|
+----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|2004|      2004|Bairro de Santa M...|              [1107]|            [Loures]|            [2004_0]|[2004_0_1, 2004_0_2]|        []|
|2729|      2729|Estação Oriente -...|        [1106, 1107]|[Parque das Naçõe...|            [2729_0]|[2729_0_1, 2729_0_2]|        []|
|2831|      2831|Casal do Bispo - ...|  [1116, 1111, 1106]|[Odivelas, Sintra...|            [2831_0]|[2831_0_1, 2831_0_2]|        []|
|3213|      3213|Pinhal de Cima - ...|              [1511]|[Sesimbra, Santan...|            [3213_0]|          [3213_0_3]|        []|
|4417|      4417|Poço Mouro - Setú...|              [1512]|[Se

In [ ]:
#show rows where id is different from short_name
lines.filter(lines.id != lines.short_name).show()

+---+----------+---------+--------------+----------+------+--------+----------+
| id|short_name|long_name|municipalities|localities|routes|patterns|facilities|
+---+----------+---------+--------------+----------+------+--------+----------+
+---+----------+---------+--------------+----------+------+--------+----------+



In [ ]:
lines_exploded = lines.select(lines.id,explode(lines.municipalities))
lines_exploded.show()

+----+----+
|  id| col|
+----+----+
|2004|1107|
|2729|1106|
|2729|1107|
|2831|1116|
|2831|1111|
|2831|1106|
|3213|1511|
|4417|1512|
|1724|1110|
|1724|1106|
|1728|1106|
|1728|1110|
|1728|1115|
|2818|1106|
|2818|1116|
|1512|1115|
|1512|1111|
|2816|1106|
|2816|1116|
|3544|1511|
+----+----+
only showing top 20 rows



In [ ]:
# Step 1: Explode array_of_ids in df1
df1_exploded = lines.withColumn("id_exploded", explode(col("municipalities")))

# Step 2: Join with df2 to get the names
df_joined = df1_exploded.join(municipalities, df1_exploded.id_exploded == municipalities.id, "left").select(df1_exploded.id, "name")

# Step 3: Group by original id and collect names into an array
df_result = df_joined.groupBy("id").agg(collect_list("name").alias("array_of_names"))

# Show the result
df_result.show()

+----+-----------------+
|  id|   array_of_names|
+----+-----------------+
|1001|        [Amadora]|
|1002|        [Amadora]|
|1003|        [Amadora]|
|1004|        [Amadora]|
|1005|        [Amadora]|
|1006|        [Amadora]|
|1008|        [Amadora]|
|1009|[Amadora, Sintra]|
|1010|        [Amadora]|
|1011|        [Amadora]|
|1012|        [Amadora]|
|1013|        [Amadora]|
|1014|        [Amadora]|
|1015|        [Amadora]|
|1101|         [Oeiras]|
|1103|         [Oeiras]|
|1105|         [Oeiras]|
|1106|         [Oeiras]|
|1107|         [Oeiras]|
|1109|         [Oeiras]|
+----+-----------------+
only showing top 20 rows



In [ ]:
municipalities.show()

+-----------+-------------+----+-------------------+------+---------+----------------+
|district_id|district_name|  id|               name|prefix|region_id|     region_name|
+-----------+-------------+----+-------------------+------+---------+----------------+
|         11|       Lisboa|1107|             Loures|    07|    PT170|             AML|
|         15|      Setúbal|1504|           Barreiro|    04|    PT170|             AML|
|         11|       Lisboa|1101|           Alenquer|    20|    PT16B|           Oeste|
|         15|      Setúbal|1511|           Sesimbra|    15|    PT170|             AML|
|         07|        Évora|0712|       Vendas Novas|    19|    PT187|Alentejo Central|
|         11|       Lisboa|1102|  Arruda dos Vinhos|    20|    PT16B|           Oeste|
|         15|      Setúbal|1510|             Seixal|    14|    PT170|             AML|
|         11|       Lisboa|1114|Vila Franca de Xira|    18|    PT170|             AML|
|         15|      Setúbal|1506|           

In [ ]:
mapping_df = municipalities["id", "name"]
mapping_df = mapping_df.withColumnRenamed("id","id_municipalities")

# Step 1: Explode the `municipalities` column
df1_exploded = lines.withColumn("id_exploded", explode(col("municipalities")))
df1_exploded.show()


+----+----------+--------------------+------------------+--------------------+--------------------+--------------------+----------+-----------+
|  id|short_name|           long_name|    municipalities|          localities|              routes|            patterns|facilities|id_exploded|
+----+----------+--------------------+------------------+--------------------+--------------------+--------------------+----------+-----------+
|2729|      2729|Estação Oriente -...|      [1106, 1107]|[Parque das Naçõe...|            [2729_0]|[2729_0_1, 2729_0_2]|        []|       1106|
|2729|      2729|Estação Oriente -...|      [1106, 1107]|[Parque das Naçõe...|            [2729_0]|[2729_0_1, 2729_0_2]|        []|       1107|
|2831|      2831|Casal do Bispo - ...|[1116, 1111, 1106]|[Odivelas, Sintra...|            [2831_0]|[2831_0_1, 2831_0_2]|        []|       1116|
|2831|      2831|Casal do Bispo - ...|[1116, 1111, 1106]|[Odivelas, Sintra...|            [2831_0]|[2831_0_1, 2831_0_2]|        []|     

In [ ]:
# Step 2: Join with mapping DataFrame to replace IDs with names
joined_df = df1_exploded.join(mapping_df, df1_exploded.id_exploded == mapping_df.id_municipalities, "left").select("id", "name")
joined_df = joined_df.drop("id_municipalities")
joined_df.show()

+----+--------+
|  id|    name|
+----+--------+
|2729|  Lisboa|
|2729|  Loures|
|2831|Odivelas|
|2831|  Sintra|
|2831|  Lisboa|
|3213|Sesimbra|
|4417| Setúbal|
|1724|  Oeiras|
|1724|  Lisboa|
|1728|  Lisboa|
|1728|  Oeiras|
|1728| Amadora|
|2818|  Lisboa|
|2818|Odivelas|
|1512| Amadora|
|1512|  Sintra|
|2816|  Lisboa|
|2816|Odivelas|
|3544|Sesimbra|
|3544|  Seixal|
+----+--------+
only showing top 20 rows



In [ ]:
result_df = joined_df.groupBy("id").agg(collect_list("name").alias("municipality_names_array"))


In [ ]:
# Step 4: Join back with the original DataFrame to include the mapped column
final_df = lines.join(result_df, "id", "left")

# Show the result
final_df["id","municipalities_name_array"].show(truncate=False)

+----+----------+----------------------------------------------------------------------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------+----------------------------------------------------------------------+----------+--------------------------------------+
|id  |short_name|long_name                                                             |municipalities          |localities                                                                                                                                     |routes                          |patterns                                                              |facilities|municipality_names_array              |
+----+----------+----------------------------------------------------------------------+------------------------+---------------------------------------------------------------

In [ ]:
final_df = final_df["id","municipality_names_array"]
final_df.show()

+----+------------------------+
|  id|municipality_names_array|
+----+------------------------+
|2729|        [Lisboa, Loures]|
|2831|    [Odivelas, Sintra...|
|3213|              [Sesimbra]|
|4417|               [Setúbal]|
|1724|        [Oeiras, Lisboa]|
|1728|    [Lisboa, Oeiras, ...|
|2818|      [Lisboa, Odivelas]|
|1512|       [Amadora, Sintra]|
|2816|      [Lisboa, Odivelas]|
|3544|      [Sesimbra, Seixal]|
|2107|                 [Mafra]|
|2740|         [Mafra, Lisboa]|
|4442|               [Setúbal]|
|4523|      [Palmela, Montijo]|
|2032|                [Loures]|
|2525|      [Loures, Odivelas]|
|3011|                [Almada]|
|4512|    [Alcochete, Monti...|
|1121|                [Oeiras]|
|2217|              [Odivelas]|
+----+------------------------+
only showing top 20 rows



In [ ]:
#select all columns from vehicles + lines.long_name (name: line_name, format:string) + municipalities.name (name: municipality_name, format: array)
vehicles_enriched = vehicles.join(lines, vehicles.line_id == lines.id, "inner").select(vehicles["*"],lines["long_name"])
#vehicles_enriched = vehicles_enriched.join(final_df, vehicles_enriched.id == final_df.id, "inner").select(vehicles_enriched["*"],final_df["municipality_names_array"])
vehicles_enriched.show()


+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|           long_name|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+
|    106|           1_1018-11|   INCOMING_AT| 41|1250| 38.73563|   1506|-9.224207|  1506_0_3|  1506_0|            SCHEDULED|        1144| 9.444445| 030074|1506_0_3_2330_235...|   173205|Amadora (Hospital...|
|    137|       ESC_DU_EU1046| IN_TRANSIT_TO| 43|2377| 38.64826|   3510|-9.177994|  3510_0_2|  3510_0|            SCHEDULED|      EU1167| 8.888889| 020612|3510_0_2_2330

In [ ]:
#vehicles_enriched = vehicles.join(lines, vehicles.line_id == lines.id, "inner").select(vehicles["*"],lines["long_name"])
vehicles_enriched = vehicles_enriched.join(final_df, vehicles_enriched.line_id == final_df.id, "left")
vehicles_enriched.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+----+------------------------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|           long_name|  id|municipality_names_array|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+----+------------------------+
|    106|           1_1018-11|   INCOMING_AT| 41|1250| 38.73563|   1506|-9.224207|  1506_0_3|  1506_0|            SCHEDULED|        1144| 9.444445| 030074|1506_0_3_2330_235...|   173205|Amadora (Hospital...|1506|       [Amadora, Oeiras]|
|    137|       ESC_DU_EU1046| IN_TRANSIT_TO| 43

In [3]:
vehicles = spark.read.parquet("/content/lake/silver/vehicles")
lines = spark.read.parquet("/content/lake/silver/lines")
municipalities = spark.read.parquet("/content/lake/silver/municipalities")

In [4]:
mapping_df = municipalities["id", "name"]
mapping_df = mapping_df.withColumnRenamed("id","id_municipalities")

df1_exploded = lines.withColumn("id_exploded", explode(col("municipalities")))
joined_df = df1_exploded.join(mapping_df, df1_exploded.id_exploded == mapping_df.id_municipalities, "left").select("id", "name")
joined_df = joined_df.drop("id_municipalities")
result_df = joined_df.groupBy("id").agg(collect_list("name").alias("municipality_names_array"))
final_df = lines.join(result_df, "id", "left")
final_df = final_df["id","municipality_names_array"]

vehicles_enriched = vehicles.join(lines, vehicles.line_id == lines.id, "inner").select(vehicles["*"],lines["long_name"])
vehicles_enriched = vehicles_enriched.join(final_df, vehicles_enriched.line_id == final_df.id, "left")



In [5]:
vehicles_enriched.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+----+------------------------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|timestamp|           long_name|  id|municipality_names_array|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+---------+--------------------+----+------------------------+
|      0|      ESC_SAB_ES2015|    STOPPED_AT| 43|2263|38.652775|   3041|-9.185986|  3041_0_2|  3041_0|            SCHEDULED|      ES2015|      0.0| 020595|3041_0_2_0900_092...|    12730|Cacilhas (Termina...|3041|                [Almada]|
|    275|20241130-64020072...| IN_TRANSIT_TO|44|

In [ ]:
df_exploded3 = vehicles_enriched.withColumn("municipality_names_array_exploded", explode(col("municipality_names_array")))

Challenge 4

In [3]:
vehicles_enriched = spark.read.parquet("/content/lake/gold/vehicles")

In [4]:
# Step 1: Explode array_of_ids in df1
df_exploded3 = vehicles_enriched.withColumn("municipality_names_array_exploded", explode(col("municipality_names_array")))
df_exploded3.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+--------------------+-----------+------------------------+---------------------------------+---------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|             trip_id|           long_name|final_df_id|municipality_names_array|municipality_names_array_exploded|timestamp|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+--------------------+--------------------+-----------+------------------------+---------------------------------+---------+
|      0|20241201-64070005...| IN_TRANSIT_TO|44|12740|38.767746|   4702|-9.100449|  4702_0_1|  4702_0|            SCHEDULED|123131000000|      0.0| 060009|4702_0_1

In [6]:
df_exploded3.count()

1453

In [8]:
if df_exploded3.count() > df_exploded3.dropDuplicates().count():
  raise ValueError('Data has duplicates')

ValueError: Data has duplicates

In [10]:
duplicate_rows = df_exploded3.exceptAll(df_exploded3.dropDuplicates())
duplicate_rows.show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+--------------------+---------+-------+--------------------+--------------------+-----------+------------------------+---------------------------------+---------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|            shift_id|    speed|stop_id|             trip_id|           long_name|final_df_id|municipality_names_array|municipality_names_array_exploded|timestamp|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+--------------------+---------+-------+--------------------+--------------------+-----------+------------------------+---------------------------------+---------+
|    311|           3_3408-31|   INCOMING_AT| 41|1264|38.687874|   1606| -9.31648|  1606_0_1|  1606_0|            SCHEDULED|               

In [12]:
df_dedup = df_exploded3.dropDuplicates()
df_dedup.count()

555

In [13]:
df_result3 = df_dedup.groupBy("municipality_names_array_exploded").count()
df_result3.show()

+---------------------------------+-----+
|municipality_names_array_exploded|count|
+---------------------------------+-----+
|                         Odivelas|   36|
|                         Barreiro|    9|
|                           Sintra|   72|
|                          Palmela|    8|
|                        Alcochete|   12|
|                          Cascais|   24|
|                            Moita|    8|
|              Vila Franca de Xira|   14|
|                          Amadora|   39|
|                           Almada|   58|
|                            Mafra|    9|
|                           Lisboa|  102|
|                           Loures|   54|
|                          Setúbal|   24|
|                          Montijo|   13|
|                         Sesimbra|    9|
|                           Oeiras|   28|
|                           Seixal|   36|
+---------------------------------+-----+



In [17]:
df_result3.sort("count", ascending=False).show()


+---------------------------------+-----+
|municipality_names_array_exploded|count|
+---------------------------------+-----+
|                           Lisboa|  102|
|                           Sintra|   72|
|                           Almada|   58|
|                           Loures|   54|
|                          Amadora|   39|
|                         Odivelas|   36|
|                           Seixal|   36|
|                           Oeiras|   28|
|                          Cascais|   24|
|                          Setúbal|   24|
|              Vila Franca de Xira|   14|
|                          Montijo|   13|
|                        Alcochete|   12|
|                         Barreiro|    9|
|                            Mafra|    9|
|                         Sesimbra|    9|
|                          Palmela|    8|
|                            Moita|    8|
+---------------------------------+-----+



In [27]:
df_result3 = df_dedup.groupBy("municipality_names_array_exploded").agg({"speed": "sum"})
df_result3.show(truncate=False)

+---------------------------------+------------------+
|municipality_names_array_exploded|sum(speed)        |
+---------------------------------+------------------+
|Odivelas                         |206.11111092567444|
|Barreiro                         |83.33333253860474 |
|Sintra                           |414.9999995827675 |
|Palmela                          |82.50000095367432 |
|Alcochete                        |138.3333339691162 |
|Cascais                          |183.3333330154419 |
|Moita                            |61.944443702697754|
|Vila Franca de Xira              |65.27777767181396 |
|Amadora                          |221.9444426894188 |
|Almada                           |394.1666693687439 |
|Mafra                            |71.66666650772095 |
|Lisboa                           |679.7222231030464 |
|Loures                           |319.7222224473953 |
|Setúbal                          |229.44444477558136|
|Montijo                          |143.3333339691162 |
|Sesimbra 

In [33]:
df_result3 = df_dedup.groupBy("municipality_names_array_exploded").agg({"speed": "avg"})
df_result3.sort("avg(speed)", ascending=False).show(truncate=False)

+---------------------------------+------------------+
|municipality_names_array_exploded|avg(speed)        |
+---------------------------------+------------------+
|Alcochete                        |11.527777830759684|
|Montijo                          |11.025641074547401|
|Palmela                          |10.31250011920929 |
|Setúbal                          |9.560185198982557 |
|Barreiro                         |9.259259170956081 |
|Mafra                            |7.962962945302327 |
|Moita                            |7.743055462837219 |
|Cascais                          |7.638888875643413 |
|Oeiras                           |7.460317435009139 |
|Sesimbra                         |7.438271681467692 |
|Almada                           |6.7959770580817915|
|Lisboa                           |6.663943363755357 |
|Seixal                           |6.3194444510671826|
|Loures                           |5.920781897173987 |
|Sintra                           |5.763888883093993 |
|Odivelas 